## Example showing generalized processing of commuting terms

## Extract Qubit-wise Commuting Groups from a Hamiltonian -  Working Example 


In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from itertools import combinations
import numpy as np
from math import sin, cos, pi
import time

# Observable Helper Functions
from observables import *
    
# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

''' another test
H = SparsePauliOp(['ZZII', 'ZIIZ', 'IZZI', 'IIZZ', 'XIII', 'IXII', 'IIIX', 'IIXI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 2.+0.j, 2.+0.j, 2.+0.j, 2.+0.j])
hamiltonian = [
    ('ZZII', 0.5),
    ('ZIIZ', 0.3),
    ('IZZI', 0.4),
    ('IIZZ', 0.2),
    ('XIII', 0.6),
    ('IXII', 0.1),
    ('IIIX', 0.7),
    ('IIXI', 0.7)
]
'''

groups = group_commuting_terms_2(hamiltonian)
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    for pauli, coeff in group:
        print(f"  {pauli}: {coeff}")



Group 1:
  XXII: 0.5
  IIZZ: 0.4
  XIII: 0.7
Group 2:
  IYYI: 0.3
  XYII: 0.2
  IIYX: 0.6
Group 3:
  IZXI: 0.1


### Create a Test Hamiltonian
Several options are provided here for testing.
We use the Ising Hamiltonian for comparison with alternative computation of energy in Observables_generalized.

In [2]:

'''
# Define the Hamiltonian terms
H_terms = [
    ('ZI', 0.5),
    ('XX', 0.3),
    ('YY', -0.1),
    #(-0.2, 'ZZ')
]

hamiltonian = H_terms
'''
# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

num_qubits = 6

###########################################

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()
    
H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
H_terms = H.to_list()

hamiltonian = H_terms

print(f"************ The test Hamiltonian")
print(hamiltonian)


************ The test Hamiltonian
[('IIIIZZ', (-0.2+0j)), ('IIIZZI', (-0.2+0j)), ('IIZZII', (-0.2+0j)), ('IZZIII', (-0.2+0j)), ('ZZIIII', (-0.2+0j)), ('IIIIIZ', (-0.4592201188381077+0j)), ('IIIIZI', (-0.4592201188381077+0j)), ('IIIZII', (-0.4592201188381077+0j)), ('IIZIII', (-0.4592201188381077+0j)), ('IZIIII', (-0.4592201188381077+0j)), ('ZIIIII', (-0.4592201188381077+0j)), ('IIIIIX', (-1.1086554390135441+0j)), ('IIIIXI', (-1.1086554390135441+0j)), ('IIIXII', (-1.1086554390135441+0j)), ('IIXIII', (-1.1086554390135441+0j)), ('IXIIII', (-1.1086554390135441+0j)), ('XIIIII', (-1.1086554390135441+0j))]


### Compute Energy of the Hamiltonian 
Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [6]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# Create circuits from the Hamiltonian directly
if not use_commuting_groups:   
    print("\n******** creating circuits from Hamltonian:")
    circuits = create_circuits_ham(num_qubits, hamiltonian)

# Convert the Hamiltonian to groups and create the circuits from the groups
else:   
    print("\n******** creating commuting groups for the Hamiltonian and circuits from the groups:")
    groups = group_commuting_terms_2(hamiltonian)
    for i, group in enumerate(groups):
        print(f"Group {i+1}:")
        for pauli, coeff in group:
            print(f"  {pauli}: {coeff}")     
    circuits = create_circuits(num_qubits, groups)

print(f"\n... constructed {len(circuits)} circuits for this Hamiltonian.")

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits, is_commuting=use_commuting_groups)

    print("")
    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating commuting groups for the Hamiltonian and circuits from the groups:
Group 1:
  IIIIZZ: (-0.2+0j)
  IIIZZI: (-0.2+0j)
  IIZZII: (-0.2+0j)
  IZZIII: (-0.2+0j)
  ZZIIII: (-0.2+0j)
  IIIIIZ: (-0.4592201188381077+0j)
  IIIIZI: (-0.4592201188381077+0j)
  IIIZII: (-0.4592201188381077+0j)
  IIZIII: (-0.4592201188381077+0j)
  IZIIII: (-0.4592201188381077+0j)
  ZIIIII: (-0.4592201188381077+0j)
Group 2:
  IIIIIX: (-1.1086554390135441+0j)
  XIIIII: (-1.1086554390135441+0j)
  IIIIXI: (-1.1086554390135441+0j)
  IIIXII: (-1.1086554390135441+0j)
  IIXIII: (-1.1086554390135441+0j)
  IXIIII: (-1.1086554390135441+0j)

... constructed 2 circuits for this Hamiltonian.

************ Compute energy of the Hamiltonian 3 times

... total execution time = 0.056
Total Energy: (-3.7011871466705624+0j)

... total execution time = 0.032
Total Energy: (-3.7141792025965024+0j)

... total execution time = 0.056
Total Energy: (-3.7293366011767657+0j)

